<a href="https://colab.research.google.com/github/Blackstocks/GenAI/blob/main/Text_Summarizer_project_Arxiv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Jun 30 14:03:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip install transformers[sentencepiece] datasets rouge_score sacrebleu py7zr -q

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.3/141.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.9/412.9 kB 32.9 MB/s eta 0:00:00


In [3]:
!pip install transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [4]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk, load_metric
import matplotlib.pyplot as plt
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
model1 = "google/pegasus-cnn_dailymail"
tokenizer1 = AutoTokenizer.from_pretrained(model1)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(model1).to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [9]:
!pip install arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.6 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=e61bca9eb349a364a5847e98cae6d5414e154daab56d22fd7768a3d7dfc14ac5
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [10]:
import arxiv

In [15]:
prompt = 'generative ai or agentic ai or machine learning or deep learning'
serch = arxiv.Search(query=prompt, max_results=10, sort_by=arxiv.SortCriterion.Relevance)

prapers = []
for result in serch.results():
    prapers.append({
        'published': result.published,
        'title': result.title,
        'abstract': result.summary,
        'authors': result.authors,
        'categories': result.categories
    })

df = pd.DataFrame(prapers)
pd.set_option('display.max_colwidth', None)
df.head(3)

,published,title,abstract,authors,categories
0,2021-07-15 22:19:15+00:00,Evaluation of Human-AI Teams for Learned and Rule-Based Agents in Hanabi,"Deep reinforcement learning has generated superhuman AI in competitive games\nsuch as Go and StarCraft. Can similar learning techniques create a superior AI\nteammate for human-machine collaborative games? Will humans prefer AI teammates\nthat improve objective team performance or those that improve subjective\nmetrics of trust? In this study, we perform a single-blind evaluation of teams\nof humans and AI agents in the cooperative card game Hanabi, with both\nrule-based and learning-based agents. In addition to the game score, used as an\nobjective metric of the human-AI team performance, we also quantify subjective\nmeasures of the human's perceived performance, teamwork, interpretability,\ntrust, and overall preference of AI teammate. We find that humans have a clear\npreference toward a rule-based AI teammate (SmartBot) over a state-of-the-art\nlearning-based AI teammate (Other-Play) across nearly all subjective metrics,\nand generally view the learning-based agent negatively, despite no statistical\ndifference in the game score. This result has implications for future AI design\nand reinforcement learning benchmarking, highlighting the need to incorporate\nsubjective metrics of human-AI teaming rather than a singular focus on\nobjective task performance.","[Ho Chit Siu, Jaime D. Pena, Edenna Chen, Yutai Zhou, Victor J. Lopez, Kyle Palko, Kimberlee C. Chang, Ross E. Allen]","[cs.AI, cs.HC]"
1,2025-02-16 23:19:44+00:00,AI Generations: From AI 1.0 to AI 4.0,"This paper proposes that Artificial Intelligence (AI) progresses through\nseveral overlapping generations: AI 1.0 (Information AI), AI 2.0 (Agentic AI),\nAI 3.0 (Physical AI), and now a speculative AI 4.0 (Conscious AI). Each of\nthese AI generations is driven by shifting priorities among algorithms,\ncomputing power, and data. AI 1.0 ushered in breakthroughs in pattern\nrecognition and information processing, fueling advances in computer vision,\nnatural language processing, and recommendation systems. AI 2.0 built on these\nfoundations through real-time decision-making in digital environments,\nleveraging reinforcement learning and adaptive planning for agentic AI\napplications. AI 3.0 extended intelligence into physical contexts, integrating\nrobotics, autonomous vehicles, and sensor-fused control systems to act in\nuncertain real-world settings. Building on these developments, AI 4.0 puts\nforward the bold vision of self-directed AI capable of setting its own goals,\norchestrating complex training regimens, and possibly exhibiting elements of\nmachine consciousness. This paper traces the historical foundations of AI\nacross roughly seventy years, mapping how changes in technological bottlenecks\nfrom algorithmic innovation to high-performance computing to specialized data,\nhave spurred each generational leap. It further highlights the ongoing\nsynergies among AI 1.0, 2.0, 3.0, and 4.0, and explores the profound ethical,\nregulatory, and philosophical challenges that arise when artificial systems\napproach (or aspire to) human-like autonomy. Ultimately, understanding these\nevolutions and their interdependencies is pivotal for guiding future research,\ncrafting responsible governance, and ensuring that AI transformative potential\nbenefits society as a whole.","[Jiahao Wu, Hengxu You, Jing Du]",[cs.AI]
2,2021-08-05 17:42:35+00:00,The AI Economist: Optimal Economic Policy Design via Two-level Deep Reinforcement Learning,"AI and reinforcement learning (RL) have improved many areas, but are not yet\nwidely adopted in economic policy design, mechanism design, or economics at\nlarge. At the same time, current economic methodology is limited by a lack of\ncounterfactual data, simplistic behavioral models, and limited opportunities to\nexperiment with policies and evaluate behavioral responses. Here we show that\nmachine-learning

In [21]:
abstract = df['abstract'][2]
abstract

'AI and reinforcement learning (RL) have improved many areas, but are not yet\nwidely adopted in economic policy design, mechanism design, or economics at\nlarge. At the same time, current economic methodology is limited by a lack of\ncounterfactual data, simplistic behavioral models, and limited opportunities to\nexperiment with policies and evaluate behavioral responses. Here we show that\nmachine-learning-based economic simulation is a powerful policy and mechanism\ndesign framework to overcome these limitations. The AI Economist is a\ntwo-level, deep RL framework that trains both agents and a social planner who\nco-adapt, providing a tractable solution to the highly unstable and novel\ntwo-level RL challenge. From a simple specification of an economy, we learn\nrational agent behaviors that adapt to learned planner policies and vice versa.\nWe demonstrate the efficacy of the AI Economist on the problem of optimal\ntaxation. In simple one-step economies, the AI Economist recovers th

In [24]:
summarizer = pipeline("summarization",model=model1, tokenizer=tokenizer1)
summarization_rtesult = summarizer(abstract)


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


In [25]:
summarization_rtesult[0]['summary_text']

'We show that machine-learning-based economic simulation is a powerful policy and mechanism design framework .<n>The AI Economist is a two-level, deep RL framework that trains both agents and a social planner who co-adapt .<n>In simple one-step economies, the AI Economist recovers the optimal tax policy of economic theory .'